In [1]:
import tsKNEE 
import scanpy as sc
import leidenalg

In [3]:
adata = sc.read_h5ad("../small_test_data/small_test_data1.h5ad")
sc.pp.neighbors(adata, use_rep='X')
sc.tl.leiden(adata)

In [4]:
print('Run Time Benchmark of Clustering')
print('tSNE')
%time sc.tl.tsne(adata)
print('tsKNEE')
%time x = tsKNEE.tsKNEE(adata)

Run Time Benchmark of Clustering
tSNE
         Falling back to preprocessing with `sc.pp.pca` and default params.
CPU times: user 3.07 s, sys: 98 ms, total: 3.17 s
Wall time: 761 ms
tsKNEE
CPU times: user 844 ms, sys: 11.9 ms, total: 855 ms
Wall time: 856 ms


In [4]:
import os
pathToMem = os.path.abspath("memusg/memusg")
os.environ['PATH'] += os.pathsep + os.path.dirname(pathToMem)

In [5]:
print('Run Time Benchmark of Plotting')
print('tSNE')
!memusg python -c "import scanpy as sc; import your_data_loading_module; adata = your_data_loading_module.load_adata(); sc.pl.tsne(adata)"
print('tsKNEE')
!memusg python -c "import tsKNEE; import your_data_loading_module; adata = your_data_loading_module.load_adata(); tsKNEE.tsKNEE_plot(adata)"

Run Time Benchmark of Plotting
tSNE
/bin/bash: memusg: command not found
tsKNEE
/bin/bash: memusg: command not found


In [6]:
print('Run Time Benchmark of Plotting')

# tSNE
print('tSNE')
!{path_to_mem}/memusg python -c "import scanpy as sc; import your_data_loading_module; adata = your_data_loading_module.load_adata(); sc.pl.tsne(adata)"

# tsKNEE
print('tsKNEE')
!{path_to_mem}/memusg python -c "import tsKNEE; import your_data_loading_module; adata = your_data_loading_module.load_adata(); tsKNEE.tsKNEE_plot(adata)"

Run Time Benchmark of Plotting
tSNE
/bin/bash: {path_to_mem}/memusg: No such file or directory
tsKNEE
/bin/bash: {path_to_mem}/memusg: No such file or directory
